In [1]:
from pathlib import Path

import jinja2
import numpy as np
import pandas as pd


# Purpose

This notebook is used to generate documentation about our sources.

In [2]:
# download data from grist
BASE_URL = "https://grist.incubateur.anct.gouv.fr/o/datainclusion/api/docs/vho3ZujRYH5vnYf2bvqTNA/download/csv"

sources_df = pd.read_csv(BASE_URL + "?tableId=Sources")
ressources_df = pd.read_csv(BASE_URL + "?tableId=Ressources")


In [5]:
DIR = Path(".")
TEMPLATE_PATH = DIR / "sources.md"
OUTPUT_PATH = DIR / "build" / "sources.md"
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

def process_grist_data(df: pd.DataFrame) -> [dict]:
    df = df.replace([np.nan], '').replace([True],'✅').replace([False],'🚫')
    if 'lien_donnees_producteurs' in df.columns:
        df['lien_donnees_producteurs'] = df['lien_donnees_producteurs'].apply(lambda x: f"[lien]({x})" if len(x) > 0 else "")
    return df.to_dict(orient="records")

# prepare template context
source_dict_list = process_grist_data(sources_df)
ressource_dict_list = process_grist_data(ressources_df)
for source_dict in source_dict_list:
    source_dict["ressources"] = [
        ressource_dict
        for ressource_dict in ressource_dict_list
        if ressource_dict["source"] == source_dict["id"]
    ]

# render template
template_loader = jinja2.FileSystemLoader(searchpath=DIR)
template_environment = jinja2.Environment(loader=template_loader)
template = template_environment.get_template(str(TEMPLATE_PATH))
with (OUTPUT_PATH).open("w") as file:
    file.write(template.render(sources=source_dict_list))